<a href="https://colab.research.google.com/github/mzndnes/Deep-Learning/blob/main/face-verification_facenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Face Verification with facenet

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import load_model

class emb:
    def __init__(self):
        self.model=load_model('./drive/My Drive/data/facenet_keras.h5',compile=False)
    def calculate(self,img):
        return self.model.predict(img)[0]

In [ ]:
def readimg(tr_t):
    # get current directory
    # curpath = os.path.abspath(os.getcwd())
    curpath='./drive/My Drive/images'
    # get directory of images to be read
    img_dir = os.path.join(curpath,  'actors_' + tr_t)
    # get list of sub directories
    dirs = os.listdir(img_dir)

    face_img = []
    lble = []
    cl = 0

    for dr in dirs:
        

        fl_dr = os.path.join(img_dir, dr)
        files = os.listdir(fl_dr)

        for fl in files:
            fl_nm = os.path.join(fl_dr, fl)
            img = cv2.imread(fl_nm, 1)
            img = cv2.resize(img, (160, 160))
            face_img.append(img)
            lble.append(cl)

        cl += 1

    # return the array of faces and corresponding labels
    return np.array(face_img), np.array(lble)


In [ ]:
import scipy.stats as stats

#incorrect value of nc may arise which is outside the valid range of [0, 5)
nc=6
dr = 'tr'
x_train, y_train = readimg(dr)
x_train_emb = []

e=emb()

for x in x_train:
    
    # img = x.astype('float') / 255.0
    img = np.expand_dims(x, axis=0)
    embs = e.calculate(img)
    x_train_emb.append(embs)
x_train_emb = np.array(x_train_emb, dtype='float')

dr = 't'
x_test, y_test = readimg(dr)
x_test_emb=[]


for x in x_test:
    # img = x.astype('float') / 255.0
    img = np.expand_dims(x, axis=0)
    embs = e.calculate(img)
    x_test_emb.append(embs)
x_test_emb = np.array(x_test_emb, dtype='float') 



In [ ]:
tp=0
tn=0
eu_dist=[]
for i in range(len(x_test_emb)):
  eu_dist.append(np.linalg.norm(x_train_emb[i]-x_test_emb[i]))
  

eu_dist=np.array(eu_dist)
eu_exp=np.exp(eu_dist)
sum_exp=np.sum(eu_exp)
eu_softmax=eu_exp/sum_exp
# print(eu_dist)
# mn=np.min(eu_dist)
# mx=np.max(eu_dist)
# print(mn)
# r=mx-mn
# eu_dist=(eu_dist-mn)/r
# eu_dist=stats.zscore(eu_dist)
print(eu_dist)

for d in eu_softmax:
  if d<0.5:
    tp=tp+1
  else:
    tn=tn+1

print('Test loss: {}, Test accuracy: {}'.format(tn/len(x_test_emb)*100, tp/len(x_test_emb) * 100))





[5.66616237 7.98689862 7.11248283 6.76714058 4.16244573 5.54223918]
Test loss: 16.666666666666664, Test accuracy: 83.33333333333334
